# Estrutura e Manipulação de GeoJSON

O GeoJSON é como um caderno de anotações digital para dados espaciais. Se um shapefile é como uma planta técnica formal, o GeoJSON é como um croqui que qualquer um pode entender e modificar facilmente!

## Objetivos
- Entender a estrutura do GeoJSON (como organizar nosso caderno)
- Criar e editar features (como fazer anotações e desenhos)
- Validar dados (como revisar nossos desenhos)
- Visualizar GeoJSON na web (como compartilhar nossos mapas online)

## 1. Configuração do Ambiente

Vamos preparar nossas ferramentas digitais:

In [ ]:
# Nossa caixa de ferramentas para trabalhar com GeoJSON:
import geopandas as gpd          # Nossa prancheta digital
import json                      # Para ler e escrever GeoJSON como texto
from shapely.geometry import Point, LineString, Polygon, shape  # Nossas ferramentas de desenho
import folium                    # Para fazer mapas web interativos
import pandas as pd              # Para organizar dados tabulares
import matplotlib.pyplot as plt   # Para visualizar nossos mapas

plt.rcParams['figure.figsize'] = (12, 8)

## 2. Estrutura do GeoJSON

### 2.1 Elementos Básicos

Um GeoJSON é como uma ficha cadastral com três partes principais:
- **type**: O tipo de registro (como o cabeçalho da ficha)
- **geometry**: O desenho em si (como um croqui anexado)
- **properties**: Informações extras (como anotações na ficha)

Vamos criar exemplos de cada tipo:

In [ ]:
# Criando um ponto (como marcar um local no mapa)
ponto_geojson = {
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [-46.6333, -23.5505]  # São Paulo
    },
    "properties": {
        "nome": "São Paulo",
        "populacao": 12325232
    }
}

# Criando uma linha (como desenhar uma estrada)
linha_geojson = {
    "type": "Feature",
    "geometry": {
        "type": "LineString",
        "coordinates": [
            [-46.6333, -23.5505],  # São Paulo
            [-43.1729, -22.9068]   # Rio de Janeiro
        ]
    },
    "properties": {
        "nome": "Rodovia Presidente Dutra",
        "extensao_km": 402
    }
}

print("Nossa ficha de ponto (como um pin no mapa):")
print(json.dumps(ponto_geojson, indent=2))

### 2.2 FeatureCollection

Uma FeatureCollection é como um álbum de fichas cadastrais:

In [ ]:
# Criando nossa coleção de features
# Como juntar várias fichas num mesmo arquivo
feature_collection = {
    "type": "FeatureCollection",
    "features": [ponto_geojson, linha_geojson]
}

# Salvando em arquivo
# Como arquivar nossas fichas
with open('../data/processed/exemplo.geojson', 'w') as f:
    json.dump(feature_collection, f, indent=2)

# Lendo com GeoPandas
# Como abrir nosso arquivo de fichas
gdf = gpd.read_file('../data/processed/exemplo.geojson')
print("\nNossos dados organizados em tabela:")
print(gdf)

## 3. Manipulação com Python

### 3.1 Criando Geometrias

In [ ]:
# Criando um polígono (como desenhar uma área)
# Área metropolitana simplificada de SP
coords = [
    [-46.8, -23.7],  # Canto inferior esquerdo
    [-46.8, -23.4],  # Canto superior esquerdo
    [-46.4, -23.4],  # Canto superior direito
    [-46.4, -23.7],  # Canto inferior direito
    [-46.8, -23.7]   # Volta ao início para fechar
]

poligono = Polygon(coords)

# Criando um GeoDataFrame
# Como organizar nossa área em uma ficha formal
gdf_poligono = gpd.GeoDataFrame(
    {'nome': ['Região Metropolitana SP'],
     'geometry': [poligono]},
    crs="EPSG:4326"
)

# Visualizando
# Como ver nosso desenho na tela
gdf_poligono.plot()
plt.title('Área Metropolitana Simplificada')
plt.show()

### 3.2 Manipulando Propriedades

In [ ]:
# Adicionando informações à nossa área
# Como preencher mais campos na ficha
gdf_poligono['area_km2'] = gdf_poligono.geometry.area * 111 * 111  # Conversão aproximada para km²
gdf_poligono['perimetro_km'] = gdf_poligono.geometry.length * 111  # Conversão aproximada para km

print("Nossa ficha completa com cálculos:")
print(gdf_poligono)

### 3.3 Visualização com Folium

In [ ]:
# Criando um mapa web
# Como preparar nosso mapa para a internet
m = folium.Map(location=[-23.5505, -46.6333], zoom_start=10)

# Adicionando nossas geometrias
# Como colocar nossos desenhos no mapa online
folium.GeoJson(gdf).add_to(m)
folium.GeoJson(feature_collection).add_to(m)

# Mostrando o mapa
m

## 4. Validação e Boas Práticas

### 4.1 Validando GeoJSON

In [ ]:
# Função para verificar se nosso GeoJSON está correto
# Como um checklist para revisar nosso trabalho
def validar_geojson(geojson_dict):
    """Checa se a estrutura básica do GeoJSON está correta"""
    try:
        if geojson_dict['type'] not in ['Feature', 'FeatureCollection']:
            return False, "Tipo de ficha inválido"
            
        if geojson_dict['type'] == 'Feature':
            if 'geometry' not in geojson_dict:
                return False, "Falta o desenho"
            if 'properties' not in geojson_dict:
                return False, "Faltam as informações"
                
        return True, "GeoJSON válido e completo"
    except Exception as e:
        return False, str(e)

# Testando nossa ficha
valido, mensagem = validar_geojson(ponto_geojson)
print(f"Resultado da revisão: {mensagem}")

### 4.2 Boas Práticas

1. **Nomenclatura**:
   - Use nomes claros (como uma boa organização de arquivo)
   - Mantenha um padrão (como um manual de preenchimento)
   
2. **Estrutura**:
   - Geometrias simples (como desenhos limpos)
   - Use FeatureCollection para grupos (como pastas organizadas)
   
3. **Coordenadas**:
   - Precisão adequada (não precisa de 10 casas decimais)
   - Verifique o sistema (como conferir a escala)
   
4. **Propriedades**:
   - Inclua metadados importantes (como legendas)
   - Mantenha tipos de dados consistentes (números com números, texto com texto)

## 5. Exercícios Práticos

Vamos praticar como um cartógrafo digital:

1. Criação de GeoJSON:
   - Marque as capitais do Sudeste no mapa
   - Adicione dados de população e área
   - Faça um mapa web bonito
   - Como fazer um atlas digital

2. Manipulação de Dados:
   - Pegue um GeoJSON existente
   - Adicione mais informações
   - Calcule áreas e distâncias
   - Como enriquecer nossos mapas

3. Conversão de Formatos:
   - Converta shapefile para GeoJSON
   - Modifique alguns dados
   - Converta de volta
   - Compare os resultados